# Import Library

In [15]:
from transformers import ViTModel, ViTFeatureExtractor
import torch.nn as nn
import torch
from transformers import ViTFeatureExtractor
from torchvision import transforms
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import torch
import torch.nn as nn

import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import MobileNet_V3_Small_Weights
from tqdm import tqdm
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, r2_score
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

In [7]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using {device} as device.")

Using cpu as device.


# Function Library

In [22]:
def train_model(model, train_loader, val_loader, y_scaler, epochs=5):
    model.to(device)

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for imgs, _, idxs in tqdm(train_loader):
            imgs = imgs.to(device)
            labels = torch.tensor([bmis_scaled[idx] for idx in idxs], dtype=torch.float32).unsqueeze(1).to(device)

            preds = model(imgs)
            loss = criterion(preds, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        print(f"\nEpoch {epoch+1}: Train Loss = {train_loss/len(train_loader):.4f}")

        # Validation
        model.eval()
        val_preds, val_labels = [], []

        with torch.no_grad():
            for imgs, _, idxs in val_loader:
                imgs = imgs.to(device)
                labels = torch.tensor(bmis_scaled[idxs], dtype=torch.float32).unsqueeze(1).to(device)

                preds = model(imgs)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_preds_real = y_scaler.inverse_transform(np.array(val_preds).reshape(-1, 1)).ravel()
        val_labels_real = y_scaler.inverse_transform(np.array(val_labels).reshape(-1, 1)).ravel()

        mae = mean_absolute_error(val_labels_real, val_preds_real)
        r2 = r2_score(val_labels_real, val_preds_real)
        r, _ = pearsonr(val_labels_real, val_preds_real)

        print(f"Val MAE: {mae:.2f} | Val R²: {r2:.3f} | Pearson r: {r:.3f}")

        # Step the scheduler
        scheduler.step(mae)

In [2]:
class ViTBMIRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.vit = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
        self.regressor = nn.Sequential(
            nn.Linear(self.vit.config.hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        outputs = self.vit(pixel_values=x)
        cls_token = outputs.last_hidden_state[:, 0]
        return self.regressor(cls_token)

In [6]:
class BMIDataset(Dataset):
    def __init__(self, csv_path, image_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        self.image_dir = image_dir
        self.transform = transform
        
        # Remove any rows without image files
        self.df['full_path'] = self.df['name'].apply(lambda x: os.path.join(image_dir, x))
        self.df = self.df[self.df['full_path'].apply(os.path.exists)].reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['full_path']).convert('RGB')

        if self.transform:
            img = self.transform(img)

        label = row['bmi']
        return img, label, idx 

# Data Loading

In [17]:
# Basic transform
img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # Normalized to [-1, 1]
])

# Dataset
dataset = BMIDataset(
    csv_path='../data/BMI/cleaned_data.csv',
    image_dir='../data/BMI/Images',
    transform=img_transform
)

# Split dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [20]:
# Load BMI column and scale
df = pd.read_csv('../data/BMI/cleaned_data.csv')
bmis = df['bmi'].values
y_scaler = StandardScaler()
bmis_scaled = y_scaler.fit_transform(bmis.reshape(-1, 1)).ravel()

In [ ]:
extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

vit_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=extractor.image_mean, std=extractor.image_std)
])

In [8]:
model = ViTBMIRegressor().to(device)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [10]:
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

# Optional learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)

In [23]:
train_model(model, train_loader, val_loader, y_scaler, epochs=10)

100%|██████████| 100/100 [07:18<00:00,  4.38s/it]



Epoch 1: Train Loss = 0.6709
Val MAE: 4.65 | Val R²: 0.388 | Pearson r: 0.647


100%|██████████| 100/100 [07:20<00:00,  4.41s/it]



Epoch 2: Train Loss = 0.4814
Val MAE: 4.45 | Val R²: 0.442 | Pearson r: 0.669


100%|██████████| 100/100 [05:59<00:00,  3.59s/it]


...


Epoch 8: Train Loss = 0.1155
Val MAE: 4.44 | Val R²: 0.470 | Pearson r: 0.688


100%|██████████| 100/100 [05:50<00:00,  3.50s/it]



Epoch 9: Train Loss = 0.1051
Val MAE: 4.53 | Val R²: 0.454 | Pearson r: 0.682


100%|██████████| 100/100 [05:48<00:00,  3.49s/it]



Epoch 10: Train Loss = 0.0955
Val MAE: 4.41 | Val R²: 0.461 | Pearson r: 0.685


In [24]:
torch.save(model.state_dict(), "vit_model_final.pt")